# Test Dickens
This notebook replicates the functionality of the `test_ollama.py` script.

In [1]:
import os
from lightrag import LightRAG, QueryParam
from lightrag.llm.openai import gpt_4o_mini_complete, gpt_4o_complete, openai_embed
from lightrag.llm.ollama import ollama_model_complete, ollama_embed
from lightrag.llm.siliconcloud import siliconcloud_embedding
from lightrag.utils import EmbeddingFunc
import asyncio
from lightrag import LightRAG, QueryParam
from lightrag.llm.openai import openai_complete_if_cache
from lightrag.llm.siliconcloud import siliconcloud_embedding
from lightrag.utils import EmbeddingFunc
import numpy as np


# Uncomment the below two lines if running in a jupyter notebook to handle the async nature of rag.insert()
import nest_asyncio
nest_asyncio.apply()

Looking in indexes: https://mirrors.cloud.tencent.com/pypi/simple


ERROR: Could not find a version that satisfies the requirement lmdeploy (from versions: none)
ERROR: No matching distribution found for lmdeploy


In [2]:
WORKING_DIR = "./KG"

if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)

# 在导入部分添加
from dotenv import load_dotenv

# 在代码的开头加载环境变量
load_dotenv()

True

In [3]:
async def llm_model_func(
    prompt, system_prompt=None, history_messages=[], keyword_extraction=False, **kwargs
) -> str:
    return await openai_complete_if_cache(
        "Qwen/Qwen2.5-7B-Instruct",
        prompt,
        system_prompt=system_prompt,
        history_messages=history_messages,
        api_key=os.getenv("SILICONFLOW_API_KEY"),
        base_url="https://api.siliconflow.cn/v1/",
        **kwargs,
    )


async def embedding_func(texts: list[str]) -> np.ndarray:
    return await siliconcloud_embedding(
        texts,
        model="netease-youdao/bce-embedding-base_v1",
        api_key=os.getenv("SILICONFLOW_API_KEY"),
        max_token_size=512,
    )



In [4]:
# function test
async def test_funcs():
    result = await llm_model_func("How are you?")
    print("llm_model_func: ", result)

    result = await embedding_func(["How are you?"])
    print("embedding_func: ", result)


asyncio.run(test_funcs())


llm_model_func:  I'm just a large language model, so I don't have feelings or emotions. How can I assist you today?
embedding_func:  [[ 3.82568017e-02  1.12520000e-02 -1.56639684e-02  1.11631686e-02
   9.46056377e-03 -5.98626165e-03  9.40134283e-03 -9.31251049e-03
  -1.66411158e-02 -1.13507025e-02 -5.73654622e-02 -9.91459191e-03
   8.42518210e-02 -7.01275980e-03 -2.97092292e-02  8.16757046e-03
   8.60185828e-03 -3.19793709e-02  1.01045927e-03  9.37173143e-03
   7.17364401e-02 -1.39860390e-02  2.33725756e-02  3.42495106e-02
   3.63617279e-02 -4.36508516e-03 -5.16703678e-03 -2.17760753e-04
  -2.05546408e-03 -9.23354924e-03 -2.51294672e-02  1.53086428e-02
   1.96317788e-02 -7.91588128e-02 -3.04593630e-02 -3.39040533e-03
  -4.94298413e-02 -1.93159338e-02  1.10703893e-01  8.81209299e-02
   2.80312989e-02  3.66380922e-02  2.40042675e-02 -3.75264063e-02
  -3.78619954e-02  9.24095220e-04  9.65796690e-03 -2.00364571e-02
   2.73009054e-02  2.68962281e-03  5.78639039e-04  2.07668506e-02
  -1.3226

In [5]:
rag = LightRAG(
    working_dir=WORKING_DIR,
    chunking_mode="markdown",
    llm_model_func=llm_model_func,
    embedding_func=EmbeddingFunc(
        embedding_dim=768, max_token_size=512, func=embedding_func
    ),
)


INFO:nano-vectordb:Load (806, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './KG/vdb_entities.json'} 806 data
INFO:nano-vectordb:Load (631, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './KG/vdb_relationships.json'} 631 data
INFO:nano-vectordb:Load (60, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './KG/vdb_chunks.json'} 60 data
INFO:lightrag:Loaded document status storage with 9 records


In [ ]:
# Choose the model here
model_choice = "ollama"  # Options: "ollama", "openai"

llm_model_func = ollama_model_complete
embedding_func = EmbeddingFunc(
    embedding_dim=1024,
    max_token_size=8192,
    func=lambda texts: ollama_embed(
        texts,
        embed_model="bge-m3:latest"
    )
)
rag = LightRAG(
    working_dir=WORKING_DIR,
    llm_model_func=llm_model_func, 
    llm_model_name='qwen2.5:7b-instruct-q4_K_M',
    embedding_func=embedding_func,
    llm_model_max_token_size=32768,
    llm_model_kwargs={"host": "http://localhost:11434", "options": {"num_ctx": 32768}},

)

In [ ]:
with open("./kg9.md") as f:
    rag.insert(f.read())

In [6]:
q = "申请登乘卡单位在登乘卡管理中负责哪些职责"

In [7]:
# Perform naive search
print("naive.....................")
print(rag.query(q, param=QueryParam(mode="naive")))

INFO:lightrag:Non-embedding cached hit(mode:naive type:query)


naive.....................
申请登乘卡单位在登乘卡管理中的主要职责包括：

1. **信息录入及日常管理**：
   - 建立严格的登乘卡请领、保管、回收、缴交等制度。
   - 负责本单位登乘卡的信息录入工作。
   - 做好登乘卡的日常管理工作。

2. **人员资质审核**：
   - 对本单位劳务工资质及相关信息进行确认。
   - 严格初审提报的通勤劳务工，确保其符合广州局集团公司劳资管理信息系统内注册的条件。
   - 通勤区间仅限劳务工工作地至广州局集团公司劳资管理信息系统登记的家庭常住地之间。

3. **动态管理和报告**：
   - 对本单位登乘卡持有者的动态管理，负责人员调入调出、离岗离职等情况的报告。


In [ ]:
# Perform local search
print("local.....................")
print(rag.query(q, param=QueryParam(mode="local")))

In [ ]:
# Perform global search
print("global.....................")
print(rag.query(q, param=QueryParam(mode="global")))

In [8]:
# Perform hybrid search
print("hybrid.....................")
print(rag.query(q, param=QueryParam(mode="hybrid")))

INFO:lightrag:Non-embedding cached missed(mode:hybrid type:query)
INFO:lightrag:Non-embedding cached missed(mode:hybrid type:keywords)


hybrid.....................


INFO:lightrag:Inserting 1 to llm_response_cache
INFO:lightrag:Query nodes: 申请登乘卡单位, 管理职责, 使用监管, 安全责任, top_k: 60, cosine: 0.2
INFO:lightrag:Query edges: 登乘卡管理, 职责, top_k: 60, cosine: 0.2
INFO:lightrag:Local query uses 60 entites, 99 relations, 5 chunks
INFO:lightrag:Global query uses 40 entites, 60 relations, 3 chunks
INFO:lightrag:Inserting 1 to llm_response_cache


根据知识库中的信息，申请登乘卡单位在登乘卡管理中主要负责以下职责：

1. **信息录入和日常管理**：负责本单位登乘卡的信息录入，并进行日常管理，建立严格的登乘卡请领、保管、回收、缴交等制度。

2. **劳资信息确认**：对本单位的劳动工资资及相关信息进行确认。只有广州局集团公司劳资管理信息系统中登记的未分配宿舍、常住地在集团管内、下班确需通勤回家的劳务工才能申请通勤登乘卡。通勤区间限定为从劳务工作地到登记的家庭常住地之间的区间。严禁超范围申请。

3. **动态管理**：对本单位登乘卡持有者的动态管理，特别是人员调入调出或离职等变动情况的及时更新。

4. **审核**：严格初审申请材料，确保申请通勤劳务工的资格符合条件。

5. **配合其他部门**：与客运部、劳卫部、财务部等单位进行协作，确保登乘卡管理工作的顺利进行。

这些职责确保了登乘卡的合理使用，并维护了登乘卡管理的规范性和安全性。


In [ ]:
# Perform mix search (Knowledge Graph + Vector Retrieval)
print("mix.....................")
print(rag.query(q, param=QueryParam(mode="mix")))